## Imports

In [113]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
import tabula

## Read-in

In [114]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all')

## Clean Data

In [115]:
df = dsf[0]
df.columns = df.iloc[0]
df = df[1:]
df = df.drop(columns=df.columns[0])
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['building_address_city'] = df['building_address_city'].str.replace('^0', '', regex=True)
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
pattern = r'^(?P<building_name>[\d\s]*[A-Za-z\s]+)\s(?P<address>.+)\s(?P<city>[A-Za-z\s]+)$'
df2[['building_name', 'address', 'city']] = df['geo_address'].str.extract(pattern, expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

## Geocode

In [116]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result
df = df.drop(columns=['address','city'])

In [117]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [118]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['geo_address'].apply(geocode)

In [119]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

## Correction section

In [120]:
## Sample Format ##
# df.at[10,'building_name']=('1500 Ocean Drive')

## Format Data

In [121]:
### Insert NaNs if needed ###
df = df.replace('N/A', np.nan)

In [122]:
df['int_Sale_Price'] = df['Sale Price'].str.replace('$','',regex=False)

In [123]:
df['int_Sale_Price'] = df['int_Sale_Price'].str.replace(',','',regex=False)

In [124]:
df['int_Sale_Price'] = pd.to_numeric(df['int_Sale_Price'])

In [125]:
df['price_per_sqft'] = df['price_per_sqft'].str.replace('$','',regex=False)
df['price_per_sqft'] = df['price_per_sqft'].str.replace(',','',regex=False)
df['price_per_sqft'] = pd.to_numeric(df['price_per_sqft'])

df['days_on_market'] = pd.to_numeric(df['days_on_market'])

## Color-code top sale

In [126]:
### Insert RANK values ###
df['RANK'] = range(1, len(df) + 1)
# use numpy to assign values to the 'COLOR' column
df['COLOR'] = np.where(df['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [127]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address', 'building_name', 'address_only',
       'geocoded', 'lat', 'lon', 'int_Sale_Price', 'RANK', 'COLOR'],
      dtype='object')

In [128]:
def popup_html(row):
    building_name = row['building_name']
    price = row['Sale Price']
    days_on_market = row['days_on_market']
    listing_agent = row['Agent']
    buyers_agent = row['Buyer Agent']
    psf = row['price_per_sqft']
    address = row['address_only']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(building_name) + '''<br>
    <strong>Sale Price: </strong>{}'''.format(price) + '''<br>
    <strong>Days on Market: </strong>{}'''.format(days_on_market) + '''<br>
    <strong>Listing Agent: </strong>{}'''.format(listing_agent) + '''<br>
    <strong>Buyer's Agent: </strong>{}'''.format(buyers_agent) + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(psf) + '''<br>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    </html>
    '''
    return html

In [129]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('June 4th - June 10th')


### Create map container ###
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['lat'], row['lon']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [130]:
m.save('index.html')

## Data snagger

In [131]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [132]:
df

,building_address_city,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,geocoded,lat,lon,int_Sale_Price,RANK,COLOR
1,Grovenor House Condo\r2627 S Bayshore Dr LPH 3...,6/15/2023,56,"$12,550,000.00",6920,1813.58,Roberta Ingletto,RGI Realty,Daniel Hertzberg,Coldwell Banker Realty,Grovenor House Condo 2627 S Bayshore Dr LPH 31...,Grovenor House Condo,2627 S Bayshore Dr LPH 3101 Miami,"(25.7322298, -80.2348919)",25.732230,-80.234892,12550000.0,1,orange
2,The Ocean Club\r791 Crandon Blvd 206\rKey Bisc...,6/15/2023,64,"$5,250,000.00",3989,1316.12,Douglas Kinsley,Fortune Christie's Int'l Real Estate,Mairym Casal,Brown Harris Stevens,The Ocean Club 791 Crandon Blvd 206 Key Biscayne,The Ocean Club,791 Crandon Blvd 206 Key Biscayne,"(25.6875627, -80.1581203)",25.687563,-80.158120,5250000.0,2,blue
3,Continuum\r50 S Pointe Dr 2004\rMiami Beach,6/12/2023,220,"$4,400,000.00",1443,3049.20,Lisa Blake,"Villazzo, LLC.",Kayce Driscoll,Douglas Elliman,Continuum 50 S Pointe Dr 2004 Miami Beach,Continuum,50 S Pointe Dr 2004 Miami Beach,"(25.7672315, -80.1327611)",25.767232,-80.132761,4400000.0,3,blue
4,Jade Signature\r16901 Collins Ave 3203\rSunny ...,6/14/2023,240,"$3,850,000.00",2511,1533.25,Gustavo Strallnikoff,Dezer Platinum Realty LLC,Viacheslav Kutsenko,"Florida Best Realty, Inc.",Jade Signature 16901 Collins Ave 3203 Sunny Is...,Jade Signature,16901 Collins Ave 3203 Sunny Isles Beach,"(25.9327557, -80.1210054)",25.932756,-80.121005,3850000.0,4,blue
5,Residences Ritz Carlton\r3400 SW 27th Ave 1002...,6/12/2023,6,"$3,000,000.00",2110,1421.80,Glen Janney,Cervera Real Estate Inc.,Glen Janney,Cervera Real Estate Inc.,Residences Ritz Carlton 3400 SW 27th Ave 1002 ...,Residences Ritz Carlton,3400 SW 27th Ave 1002 Coconut Grive,"(25.7297273, -80.2378425)",25.729727,-80.237842,3000000.0,5,blue
6,Icon South Beach\r450 Alton Rd 3202\rMiami Beach,6/12/2023,35,"$2,800,000.00",1933,1448.53,Sean Murray,Douglas Elliman,Judith Zeder,Coldwell Banker Realty,Icon South Beach 450 Alton Rd 3202 Miami Beach,Icon South Beach,450 Alton Rd 3202 Miami Beach,"(25.7736596, -80.1411255)",25.773660,-80.141126,2800000.0,6,blue
7,Four Seasons\r1425 Brickell Ave 45B\rMiami,6/16/2023,414,"$2,425,000.00",2114,1147.11,Magin Hernandez,Magin New World Realty,Vanessa Frank PA,One Sotheby's International Realty,Four Seasons 1425 Brickell Ave 45B Miami,Four Seasons,1425 Brickell Ave 45B Miami,"(25.7591323, -80.1918295)",25.759132,-80.191829,2425000.0,7,blue
8,900 Biscayne Bay\r900 Biscayne Blvd PH-6109\rM...,6/16/2023,42,"$1,875,000.00",2577,727.59,Benjamin Moss,"Compass Florida, LLC",Andrea Da Silva,"Related ISG Realty, LLC.",900 Biscayne Bay 900 Biscayne Blvd PH-6109 Miami,900 Biscayne Bay,900 Biscayne Bay 900 Biscayne Blvd PH-6109 Miami,"(25.7834936, -80.19020909999999)",25.783494,-80.190209,1875000.0,8,blue
9,Ocean Two\r19111 Collins Ave 1004\rSunny Isles...,6/14/2023,167,"$1,825,000.00",2440,747.95,Jocelyn Alter Furman,"BMore Group, LLC",Joanna Jimenez,"Compass Florida, LLC",Ocean Two 19111 Collins Ave 1004 Sunny Isles B...,Ocean Two,19111 Collins Ave 1004 Sunny Isles Beach,"(25.9537166, -80.1196968)",25.953717,-80.119697,1825000.0,9,blue
10,Ocean Two\r19111 Collins Ave 2005\rSunny Isles...,6/12/2023,69,"$1,800,000.00",2440,737.70,Wendy Cohen,One Sotheby's International Realty,Daniel Cohen,One Sotheby's International Realty,Ocean Two 19111 Collins Ave 2005 Sunny Isles B...,Ocean Two,19111 Collins Ave 2005 Sunny Isles Beach,"(25.9537166, -80.1196968)",25.953717,-80.119697,1800000.0,10,blue


In [133]:
### Highest and lowest sale price ###
print(f"{ME}{BR}{df.loc[df['int_Sale_Price'].idxmax()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmax()]['address_only']} | Price ${df.loc[df['int_Sale_Price'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmax()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmax()]['days_on_market']}")
print(f"{LE}{BR}{df.loc[df['int_Sale_Price'].idxmin()]['building_name']}, {df.loc[df['int_Sale_Price'].idxmin()]['address_only']} | Price ${df.loc[df['int_Sale_Price'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['int_Sale_Price'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['int_Sale_Price'].idxmin()]['Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Agent']} with {df.loc[df['int_Sale_Price'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['int_Sale_Price'].idxmin()]['days_on_market']}")
### Highest and lowest psf ###
print(f"{MAX_PSF}{BR}{df.loc[df['price_per_sqft'].idxmax()]['building_name']}, {df.loc[df['price_per_sqft'].idxmax()]['address_only']} | Price ${df.loc[df['price_per_sqft'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmax()]['Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmax()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmax()]['days_on_market']}")
print(f"{MIN_PSF}{BR}{df.loc[df['price_per_sqft'].idxmin()]['building_name']}, {df.loc[df['price_per_sqft'].idxmin()]['address_only']} | Price ${df.loc[df['price_per_sqft'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['price_per_sqft'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['price_per_sqft'].idxmin()]['Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['price_per_sqft'].idxmin()]['Buyer Agent']} with {df.loc[df['price_per_sqft'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['price_per_sqft'].idxmin()]['days_on_market']}")
### Highest and lowest days on market ###
print(f"{DAYS_MAX}{BR}{df.loc[df['days_on_market'].idxmax()]['building_name']}, {df.loc[df['days_on_market'].idxmax()]['address_only']} | Price ${df.loc[df['days_on_market'].idxmax()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmax()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmax()]['Agent']} with {df.loc[df['days_on_market'].idxmax()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmax()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmax()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmax()]['days_on_market']}")
print(f"{DAYS_MIN}{BR}{df.loc[df['days_on_market'].idxmin()]['building_name']}, {df.loc[df['days_on_market'].idxmin()]['address_only']} | Price ${df.loc[df['days_on_market'].idxmin()]['int_Sale_Price']:,.0f} | ${df.loc[df['days_on_market'].idxmin()]['price_per_sqft']:,.0f} psf | Listing agent: {df.loc[df['days_on_market'].idxmin()]['Agent']} with {df.loc[df['days_on_market'].idxmin()]['Listing Broker']} | Buyer's agent: {df.loc[df['days_on_market'].idxmin()]['Buyer Agent']} with {df.loc[df['days_on_market'].idxmin()]['Buyer Broker']} | Days on market: {df.loc[df['days_on_market'].idxmin()]['days_on_market']}")

Most Expensive
Grovenor House Condo, 2627 S Bayshore Dr LPH 3101 Miami | Price $12,550,000 | $1,814 psf | Listing agent: Roberta Ingletto with RGI Realty | Buyer's agent: Daniel Hertzberg with Coldwell Banker Realty | Days on market: 56
Least Expensive
Ocean Two, 19111 Collins Ave 2005 Sunny Isles Beach | Price $1,800,000 | $738 psf | Listing agent: Wendy Cohen with One Sotheby's International Realty | Buyer's agent: Daniel Cohen with One Sotheby's International Realty | Days on market: 69
Highest Price Per Square Foot
Continuum, 50 S Pointe Dr 2004 Miami Beach | Price $4,400,000 | $3,049 psf | Listing agent: Lisa Blake with Villazzo, LLC. | Buyer's agent: Kayce Driscoll with Douglas Elliman | Days on market: 220
Lowest Price Per Square Foot
900 Biscayne Bay, 900 Biscayne Bay 900 Biscayne Blvd PH-6109 Miami | Price $1,875,000 | $728 psf | Listing agent: Benjamin Moss with Compass Florida, LLC | Buyer's agent: Andrea Da Silva with Related ISG Realty, LLC. | Days on market: 42
Most Days 

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [134]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [135]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_06202023
